# **RECORDLINKAGE  TOOL**

### Required imports

In [1]:
import pandas as pd
import numpy as np
import recordlinkage

from recordlinkage import datasets


### Loading datasets 

In [2]:
first_febrl_dataset, second_febrl_dataset = datasets.load_febrl4(return_links=False)

In [3]:
indexes_frst_febrl_dataset = sorted(first_febrl_dataset)
indexes_scd_febrl_dataset = sorted(second_febrl_dataset.index)
true_links = list(zip(indexes_frst_febrl_dataset, indexes_scd_febrl_dataset))

In [4]:
first_febrl_dataset.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


In [5]:
second_febrl_dataset.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


**PREPROCESSING**

**INDEXING STEP**

L'indexation n'est faisable que sur un ou deux jeu de données. Il propose 4 méthodes d'indexations:
- Full : renvoie toutes les combinaisons de paires possibles
- Block : renvoie tous les éléments qui concordent par rapport aux variables données en entrée 
- SortedNeighbourhood : renvoie tous les éléments qui concordent par rapport aux variables données en entrée et celles dans leur voisinnage
- Random : renvoi des paires crées aléatoirement

Ici, nous testons la méthode du block en utilisant les colonnes 'given_name' et 'address_1'. Avec le blocage sur le nom et adresse, les candidats seront filtrés pour n'inclure que ceux dont les valeurs sont similaires.

In [6]:
indexer = recordlinkage.Index()
indexer.block(on='given_name')
indexer.block(on='address_1')
candidate_pairs = indexer.index(first_febrl_dataset, second_febrl_dataset)

Dans le cas ou les données sont susceptible de contenir des erreurs de typographies, utiliser la méthode SortedNeighborhood peut ajouter une certaine flexibilité pour les fautes d'orthographe mineures. 

**COMPARAISON**

La méthode suivante consiste à comparer les paires en utilisant Compare. </br>
Nous pouvons définir plusieurs options pour la façon dont nous voulons comparer les colonnes de données.

In [7]:
comp = recordlinkage.Compare()

# initialise similarity measurement algorithms
comp.string('given_name', 'given_name', method='jarowinkler')
comp.string('surname', 'surname', method='jarowinkler')
comp.string('address_1', 'address_1', method='levenshtein')
comp.exact('soc_sec_id', 'soc_sec_id')

# the method .compute() returns the DataFrame with the feature vectors.
features = comp.compute(candidate_pairs, first_febrl_dataset, second_febrl_dataset)

**CLASSIFICATION**

Les vecteurs de scores de similarités sont donnés en entrée au classifieur (supervisé ou non supervisé). 

In [8]:
kmeans_clf = recordlinkage.KMeansClassifier()
links_pred_kmeans = kmeans_clf.fit_predict(features).values